In [17]:
import os, errno, sys
import math
import logging
import numpy as np
import tensorflow as tf
from keras import backend as K
from keras.layers import Dense
from keras.models import Model 
from keras.optimizers import Adam, SGD, RMSprop
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from keras.applications.densenet import DenseNet121
from keras.applications.nasnet import NASNetLarge
from sklearn.utils.class_weight import compute_class_weight, compute_sample_weight
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from keras.applications import ResNet101V2 
from keras.callbacks import  ModelCheckpoint
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import roc_curve, auc
from numpy import interp
from itertools import cycle

In [ ]:
X = np.load("/content/drive/MyDrive/Colab Notebooks/Dataset/np_data/Test Numpy.npy",allow_pickle=True)
y = np.load("/content/drive/MyDrive/Colab Notebooks/Dataset/np_data/SECOND TRAIN_label.npy",allow_pickle=True)

X_test = np.load("/content/drive/MyDrive/Colab Notebooks/Dataset/np_data/test folder not the training.npy",allow_pickle=True)
y_test = np.load("/content/drive/MyDrive/Colab Notebooks/Dataset/np_data/SECOND TEST_label.npy",allow_pickle=True)

X = np.array([np.array(fold) for fold in X])
y = np.array([np.array(fold) for fold in y])
X_test = np.array(X_test)
y_test = np.array(y_test)

In [18]:
def calculateFprTprAuc(binary_y_test, y_score):
    # Compute ROC curve and ROC area for each class
    n_classes = 3
    
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(binary_y_test[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(binary_y_test.ravel(), y_score.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    return fpr , tpr , roc_auc

In [19]:
def plotROC(fpr,tpr,roc_auc,title):
    #aggregate all false positive rates
    classes_list = np.array(["Glaucoma","Normal","Other"])
    lw = 2
    n_classes = 3
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

    #interpolate all ROC curves
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += interp(all_fpr, fpr[i], tpr[i])

    #average tpr and compute AUC
    mean_tpr /= n_classes

    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

    # Plot all ROC curves
    plt.figure()
    plt.plot(fpr["micro"], tpr["micro"],
             label='micro-average ROC curve (area = {0:0.2f})'
                   ''.format(roc_auc["micro"]),
             color='deeppink', linestyle=':', linewidth=4)

    plt.plot(fpr["macro"], tpr["macro"],
             label='macro-average ROC curve (area = {0:0.2f})'
                   ''.format(roc_auc["macro"]),
             color='navy', linestyle=':', linewidth=4)

    colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
    for i, color in zip(range(n_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=lw,
                 label='ROC curve of {0} (area = {1:0.2f})'
                 ''.format(classes_list[i], roc_auc[i]))

    plt.plot([0, 1], [0, 1], 'k--', lw=lw)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.show()

In [20]:
########## plot confusion matrix
#class_list must be numpy array

def plot_confusion_matrix(y_true, y_pred, classes,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    np.set_printoptions(precision=2)
    
    if not title:
        title = 'Normalized confusion matrix'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    #classes = classes[unique_labels(y_true, y_pred)]
    
    cm_normalize = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print("Normalized confusion matrix")

    #print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm_normalize, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm_normalize.shape[1]),
           yticks=np.arange(cm_normalize.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' #if normalize else 'd'
    thresh = cm_normalize.max() / 2.
    for i in range(cm_normalize.shape[0]):
        for j in range(cm_normalize.shape[1]):
            ax.text(j, i, "{:.2%}\n({:d})".format(cm_normalize[i, j], cm[i, j]),
                    ha="center", va="center",
                    color="white" if cm_normalize[i, j] > thresh else "black")
    fig.tight_layout()
    #plt.xlim(-0.5, len(np.unique(y))-0.5)
    #plt.ylim(len(np.unique(y))-0.5, -0.5)
    plt.show()
    
    
    FP = cm.sum(axis=0) - np.diag(cm)  
    FN = cm.sum(axis=1) - np.diag(cm)
    TP = np.diag(cm)
    TN = cm.sum() - (FP + FN + TP)

    FP = FP.astype(float)
    FN = FN.astype(float)
    TP = TP.astype(float)
    TN = TN.astype(float)
    
    # Sensitivity, hit rate, recall, or true positive rate
    TPR = TP/(TP+FN)
    # Specificity or true negative rate
    TNR = TN/(TN+FP) 
    # Precision or positive predictive value
    PPV = TP/(TP+FP)
    # Negative predictive value
    NPV = TN/(TN+FN)
    # Fall out or false positive rate
    FPR = FP/(FP+TN)
    # False negative rate
    FNR = FN/(TP+FN)
    # False discovery rate
    FDR = FP/(TP+FP)
    # F1-score
    F1 = 2*((PPV*TPR)/(PPV+TPR))
    # Overall accuracy
    ACC = (TP+TN)/(TP+FP+FN+TN)

    print("="*60)
    print("{:<26} | {:>8} {:>8} {:>8}".format("value", "Glaucoma", "Normal", "Other"))
    print("="*60)
    print("{:<26} | {:>8.2f} {:>8.2f} {:>8.2f}".format("Sensitivity(TPR)", TPR[0], TPR[1], TPR[2]))
    print("{:<26} | {:>8.2f} {:>8.2f} {:>8.2f}".format("Specificity(TNR)", TNR[0], TNR[1], TNR[2]))
    print("{:<26} | {:>8.2f} {:>8.2f} {:>8.2f}".format("Precision(PPV)", PPV[0], PPV[1], PPV[2]))
    print("{:<26} | {:>8.2f} {:>8.2f} {:>8.2f}".format("Negative predictive value", NPV[0], NPV[1], NPV[2]))
    print("{:<26} | {:>8.2f} {:>8.2f} {:>8.2f}".format("False positive rate", FPR[0], FPR[1], FPR[2]))
    print("{:<26} | {:>8.2f} {:>8.2f} {:>8.2f}".format("False negative rate", FNR[0], FNR[1], FNR[2]))
    print("{:<26} | {:>8.2f} {:>8.2f} {:>8.2f}".format("False discovery rate", FDR[0], FDR[1], FDR[2]))
    print("{:<26} | {:>8.2f} {:>8.2f} {:>8.2f}".format("F1-score", F1[0], F1[1], F1[2]))
    print("{:<26} | {:>8.2f} {:>8.2f} {:>8.2f}".format("Overall accuracy", ACC[0], ACC[1], ACC[2]))



In [21]:
n_fold = 5
height, width = 256, 256
batch_size = 32
epochs = 50
num_output = 3
base_lr = 0.0001
optimizers = {'adam': Adam(lr=base_lr),
					 'sgd' : SGD(lr=base_lr, decay=1e-6, momentum=0.9, nesterov=True),
					 'rms' : RMSprop(lr=base_lr, rho=0.9, epsilon=1e-08, decay=0.0)
					}

In [22]:
def build_model(base_model_name):
    
    print('Creating Model : {}.......'.format(base_model_name.upper()))
    if base_model_name == 'DenseNet121' :
        init_model = DenseNet121(input_shape= (height, width ,3), include_top=False, weights='imagenet', pooling='avg')
        x = init_model.output
        out_layer = Dense(num_output, activation="softmax")(x)
        model = Model(init_model.input, out_layer)
    elif base_model_name == 'ResNet101V2' :
        init_model = ResNet101V2(input_shape= (height, width ,3), include_top=False, weights='imagenet', pooling='avg')
        x = init_model.output
        out_layer = Dense(num_output, activation="softmax")(x)
        model = Model(init_model.input,out_layer)
    else :
        sys.exit("Model Not Support")
    return	model

In [23]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')

reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')

In [ ]:
kf = KFold(n_splits=5)
kf.get_n_splits(X)
bestFold = 0
bestModel = None
best_sum_average_AUC = 0
best_sum_3class_AUC = 0
for i ,(train_index, val_index) in enumerate(kf.split(X)):
    
    model_name = "DenseNet121" 
    model = build_model(model_name)
    #model.summary()
    
    #class_weights = get_class_weights(y_train[train_idx])
    model.compile(optimizer=optimizers['adam'], loss='categorical_crossentropy', metrics=['accuracy'])
    
    checkpoint_path =  "/content/drive/MyDrive/Colab Notebooks/Dataset/weight/{}-weights_fold-{}.hdf5".format(model_name,i+1)
    mcp_save = ModelCheckpoint(checkpoint_path, save_best_only=True, monitor='val_loss', mode='min',save_weights_only=True)
    
    X_train = np.concatenate(X[train_index],axis=0)
    X_val = X[val_index][0]
    y_train = np.concatenate(y[train_index],axis=0)
    y_val = y[val_index][0]

    y_train = to_categorical(y_train,3)
    y_val = to_categorical(y_val,3)

    
    print('Training model.....')
    # train_datagen = ImageDataGenerator( #samplewise_center=True,
    #     # rotation_range=90,
    #     # width_shift_range=0.2,
    #     # height_shift_range=0.2,
    #     #zoom_range=0.2,
    #     # horizontal_flip=True,
    #     # vertical_flip=True,
    #     # fill_mode = 'constant'
    #     )

    # train_datagen.fit(X_train, seed=1)
    # train_gen =  train_datagen.flow(X_train, y_train, batch_size=batch_size)
    # # 	train_crops = crop_generator(train_gen, 224)

    # history = model.fit(train_gen,
    #     steps_per_epoch = (len(X_train) / batch_size ), 
    #     epochs = epochs,
    #     validation_data=(X_train, y_valid),
    #     callbacks = cp_callback
    #     )
    history = model.fit(X_train,y_train,
      steps_per_epoch = (len(X_train) / batch_size ), 
      epochs = 50,
      validation_data=(X_val, y_val),
      callbacks=[earlyStopping, mcp_save, reduce_lr_loss]
      )
    
    
    binary_y_val = label_binarize(y_val, classes=[0, 1, 2])

    fpr, tpr, roc_auc = calculateFprTprAuc(binary_y_val, model.predict(X_val))
    plotROC(fpr,tpr,roc_auc,"ROC curve fold-"+str(i+1))


    if roc_auc["micro"]+roc_auc["macro"] > best_sum_average_AUC:
        best_sum_average_AUC = roc_auc["micro"]+roc_auc["macro"]
        best_sum_3class_AUC = roc_auc[0] + roc_auc[1] + roc_auc[2]
        bestModel = model
        bestFold = i+1
    elif roc_auc["micro"]+roc_auc["macro"] == best_sum_average_AUC and best_sum_3class_AUC < roc_auc[0] + roc_auc[1] + roc_auc[2]:
        best_sum_average_AUC = roc_auc["micro"]+roc_auc["macro"]
        best_sum_3class_AUC = roc_auc[0] + roc_auc[1] + roc_auc[2]
        bestModel = model
        bestFold = i+1
print("best model is fold",bestFold)
    




In [ ]:
binary_y_test = label_binarize(y_test, classes=[0, 1, 2])
fpr, tpr, roc_auc = calculateFprTprAuc(binary_y_test, bestModel.predict(X_test))
plotROC(fpr,tpr,roc_auc,"ROC curve of best model with test set")

In [ ]:
classes_list = ["Glaucoma","Normal","Other"]
display_labels = classes_list
y_prob = bestModel.predict(X_test)
y_pred = np.argmax(y_prob, axis=1)

plot_confusion_matrix(y_test, y_pred, classes_list, title="Confusion Metric")

